In [177]:
import pandas as pd
import tensorflow as tf
import re
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk.translate.bleu_score as bleu
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
#%run pre_processing.ipynb
#import pre_processing
!pip install import-ipynb
import import_ipynb
import pre_processing



In [178]:
#Reading the dataset
df=pd.read_csv('/content/drive/MyDrive/Pre_processing.csv')
eng_tel=shuffle(df)
#eng_tel.shape

In [179]:
eng_tel.drop('Unnamed: 0', axis=1, inplace=True)

In [180]:
eng_tel.head()

,Telugu,English
1246,<start> ఆధునిక పార్లమెంటరీ ప్రజాస్వామ్య దేశాల ...,<start> some of the salient features in the fu...
2096,<start> అతను మండుతున్న వ్యక్తి మరియు రాజుకు కూ...,<start> he was a fiery man an was not afrai of...
4631,<start> విమర్శ <end>,<start> criticism <end>
5163,<start> కమిటీ యొక్క విధులు భారత ప్రభుత్వంతో సం...,<start> the functions of the committee are to ...
2053,<start> నేను యుఎస్ వెళ్ళే ముందు నేను చెప్పలేదు...,<start> i must say that before i went to the u...


Tokensation: 

In [181]:
def tokenize(lang):

  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post',maxlen=20,dtype='int32')

  return tensor, lang_tokenizer

In [182]:
def load_dataset():

  input_tensor, inp_lang_tokenizer = tokenize(eng_tel['English'].values)
  target_tensor, targ_lang_tokenizer = tokenize(eng_tel['Telugu'].values)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [183]:
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset()

In [184]:
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

## Train_Test Spilt: 

In [185]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.10)

Model Building Requriments:

In [186]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 16
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 128
units = 1024
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE

vocab_inp_size =len(inp_lang.word_index.keys())
#print(vocab_inp_size)
vocab_tar_size =len(targ_lang.word_index.keys())

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

Glove Usage for English Vocabulary:


In [187]:
# Global Vector Which is Glove Which give very word in 100 vectors

embeddings_index = dict()
f = open('/content/drive/MyDrive/glove.6B.100d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    #print(word)
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    #print(embeddings_index[word])
f.close()

embedding_matrix = np.zeros((vocab_inp_size+1, 100))
for word, i in inp_lang.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Encoder:


In [188]:
# Encoder Declaration: Declaring Encoder which we passing the vectors to the encoder here I am using GRU(Gated recurrent units)
#Reference from Internet

class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, name="embedding_layer_encoder",trainable=False)
        self.gru = tf.keras.layers.GRU(units, return_sequences=True, return_state=True, recurrent_activation='sigmoid', recurrent_initializer='glorot_uniform')
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

Decoder:

In [189]:
# Decoding  
#Reference from Internet

class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(units, return_sequences=True, return_state=True, recurrent_activation='sigmoid', recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # Bahaudan Attention layer
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, x, hidden, enc_output):

        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        attention_weights = tf.nn.softmax(score, axis=1)
        
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        x = self.embedding(x)
        
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        output, state = self.gru(x)
        
        output = tf.reshape(output, (-1, output.shape[2]))
        
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [190]:
#Calling of Encoders and Decoders:

In [191]:
#tf.keras.backend.clear_session()

encoder = Encoder(vocab_inp_size+1, 100, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size+1, embedding_dim, units, BATCH_SIZE)

Loss Function:


In [192]:
# Loss Function:

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction='none')


def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

#Passing Value into the Function:


In [193]:

@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)
    encoder.get_layer('embedding_layer_encoder').set_weights([embedding_matrix])
    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    for t in range(1, targ.shape[1]):
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [194]:
EPOCHS =5

for epoch in range(EPOCHS):
  #start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print(f'Epoch {epoch+1} Batch {batch} Loss {batch_loss.numpy():.4f}')

  print(f'Previous Loss {total_loss/steps_per_epoch:}')

Epoch 1 Batch 0 Loss 5.4194
Epoch 1 Batch 100 Loss 4.4580
Epoch 1 Batch 200 Loss 4.6898
Epoch 1 Batch 300 Loss 3.6412
Previous Loss 4.401186943054199
Epoch 2 Batch 0 Loss 4.9686
Epoch 2 Batch 100 Loss 5.6720
Epoch 2 Batch 200 Loss 3.3125
Epoch 2 Batch 300 Loss 3.5197
Previous Loss 4.091197490692139
Epoch 3 Batch 0 Loss 3.9615
Epoch 3 Batch 100 Loss 4.8506
Epoch 3 Batch 200 Loss 3.8877
Epoch 3 Batch 300 Loss 4.5092
Previous Loss 3.961367607116699
Epoch 4 Batch 0 Loss 3.3436
Epoch 4 Batch 100 Loss 3.2262
Epoch 4 Batch 200 Loss 3.8686
Epoch 4 Batch 300 Loss 3.8844
Previous Loss 3.824436902999878
Epoch 5 Batch 0 Loss 4.3109
Epoch 5 Batch 100 Loss 4.1379
Epoch 5 Batch 200 Loss 3.1820
Epoch 5 Batch 300 Loss 4.1687
Previous Loss 3.6572601795196533


In [195]:
import string
def preprocess(text):
    text = text.lower() # lower casing
    exclude = set(string.punctuation) # Set of all special characters
    text = re.sub("'", '', text) # remove the quotation marks if any
    text = ''.join(ch for ch in text if ch not in exclude)
    text = text.strip()
    text = re.sub(" +", " ", text) # remove extra spaces
    text = '<start> ' + text + ' <end>'
    return text

Main Function

In [196]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],maxlen=20, padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)
    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()
    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result,attention_plot

    dec_input = tf.expand_dims([predicted_id], 0)

  return result,attention_plot

Testing:

In [217]:
input_sentence=input("Enter something to translate:")
print('Input sentence in english : ',input_sentence)
predicted_output,attention_plot=evaluate(input_sentence)
print('Predicted sentence in telugu : ',predicted_output)

Enter something to translate:you
Input sentence in english :  you
Predicted sentence in telugu :  మీరు ధన్యవాదాలు <end> 


In [198]:
#import pickle
#file = open('list.pkl', 'wb')
# dump information to the file

#pickle.dump(evaluate(), file)
# close the file
#file.close()


In [199]:
l =predicted_output
split_list =[i.split() for i in l]
print(split_list)

[['మ'], ['ీ'], ['ర'], ['ు'], [], ['ధ'], ['న'], ['్'], ['య'], ['వ'], ['ా'], ['ద'], ['ా'], ['ల'], ['ు'], [], ['<'], ['e'], ['n'], ['d'], ['>'], []]


In [200]:
#Bleu score to find accuracy of the model
from nltk.translate.bleu_score import sentence_bleu
candidate = 'మీరు'.split()
print('BLEU score -> {}'.format(sentence_bleu(split_list, candidate)))

BLEU score -> 0
